In [1]:
import numpy as np # linear algebra
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [2]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
train_data = pd.read_csv(r"C:\Users\user\Downloads\titanic\train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data = pd.read_csv(r"C:\Users\user\Downloads\titanic\test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
y= train_data["SibSp"]

In [10]:
#Features
features=["Sex", "Pclass", "Parch", "Fare", "Embarked", "Age"]

In [11]:
train_data["Age"] = train_data["Age"].fillna(train_data["Age"].median())
test_data["Age"] = test_data["Age"].fillna(test_data["Age"].median())

train_data["Fare"] = train_data["Fare"].fillna(train_data["Fare"].median())
test_data["Fare"] = test_data["Fare"].fillna(test_data["Fare"].median())

train_data["Embarked"] = train_data["Embarked"].fillna(train_data["Embarked"].mode()[0])
test_data["Embarked"] = test_data["Embarked"].fillna(test_data["Embarked"].mode()[0])

In [12]:
# Make sure this is your training data (891 rows)
X = pd.get_dummies(train_data[["Sex", "Pclass", "Parch", "Fare", "Embarked", "Age"]])
X_test = pd.get_dummies(test_data[["Sex", "Pclass", "Parch", "Fare", "Embarked", "Age"]])

# Align test columns to train
X, X_test = X.align(X_test, join='left', axis=1, fill_value=0)

# Set your target variable from the training set only!
y = train_data["SibSp"]

# Fit cross_val_score on X and y (same number of rows: 891)
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
rf_scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')
rf_acc = rf_scores.mean()
print(f"Random Forest CV Accuracy: {rf_acc:.4f}")

Random Forest CV Accuracy: 0.7509


In [16]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Encode y as labels (only needed if y has categorical type but is represented numerically)
y_class = LabelEncoder().fit_transform(y)

xgb_model = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1,
                          use_label_encoder=False, eval_metric='mlogloss', random_state=1)

xgb_scores = cross_val_score(xgb_model, X, y_class, cv=5, scoring='accuracy')
xgb_acc = xgb_scores.mean()

print(f"XGBoost Classifier CV Accuracy: {xgb_acc:.4f}")



C:\Users\user\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [15:56:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\user\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [15:56:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\user\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [15:56:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\user\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [15:56:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:7

XGBoost Classifier CV Accuracy: 0.8047


In [17]:
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

# Encode SibSp if needed (ensures it's treated as classification labels)
y_class = LabelEncoder().fit_transform(y)

# Create and evaluate SVM model
svm_model = SVC(kernel='rbf', C=1, gamma='scale', random_state=1)

# Perform cross-validation
svm_scores = cross_val_score(svm_model, X, y_class, cv=5, scoring='accuracy')
svm_acc = svm_scores.mean()

print(f"SVM Classifier CV Accuracy: {svm_acc:.4f}")


SVM Classifier CV Accuracy: 0.6981


In [21]:

# Encode the target variable
y_class = LabelEncoder().fit_transform(y)

# Define models
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
xgb_model = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1,
                          use_label_encoder=False, eval_metric='mlogloss', random_state=1)
svm_model = SVC(kernel='rbf', C=1, gamma='scale', random_state=1)

# Evaluate models
rf_acc = cross_val_score(rf_model, X, y_class, cv=5, scoring='accuracy').mean()
xgb_acc = cross_val_score(xgb_model, X, y_class, cv=5, scoring='accuracy').mean()
svm_acc = cross_val_score(svm_model, X, y_class, cv=5, scoring='accuracy').mean()

# Print model accuracies
print(f"Random Forest Accuracy: {rf_acc:.4f}")
print(f"XGBoost Accuracy: {xgb_acc:.4f}")
print(f"SVM Accuracy: {svm_acc:.4f}")

# Compare using if-elif-else
if (rf_acc > xgb_acc) and (rf_acc > svm_acc):
    print("\nBest Model: Random Forest")
    print("Your submission was successfully saved!")
elif (xgb_acc > rf_acc) and (xgb_acc > svm_acc):
    print("\nBest Model: XGBoost")
    print("Your submission was successfully saved!")
elif (svm_acc > rf_acc) and (svm_acc > xgb_acc):
    print("\nBest Model: SVM")
    print("Your submission was successfully saved!")
else:
    print("\nThere is a tie between models.")
    print("Your submission was successfully saved!")


C:\Users\user\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [16:04:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\user\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [16:04:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\user\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [16:04:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\user\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [16:04:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:7

Random Forest Accuracy: 0.7509
XGBoost Accuracy: 0.8047
SVM Accuracy: 0.6981

Best Model: XGBoost
Your submission was successfully saved!
